In [3]:
import train
import torch
import torchvision
import dataset
import time
import copy
import tqdm

import albumentations
from albumentations.pytorch.transforms import ToTensorV2
import segmentation_models_pytorch as smp
from torch.utils.tensorboard import SummaryWriter
import logging

console = logging.StreamHandler()
#console_formatter = logging.Formatter('[%(levelname)s] [%(name)s: %(funcName)s] %(message)s')
#console.setFormatter(console_formatter)
#console.setLevel(logging.INFO)

logging.basicConfig(format='[%(levelname)s] [%(name)s: %(funcName)s] %(message)s')

log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)
#logging.getLogger(__name__).addHandler(console)


In [5]:
    data_transforms = albumentations.Compose([
        albumentations.Flip(),
        albumentations.RandomBrightness(0.2),
        albumentations.ShiftScaleRotate(rotate_limit=90, scale_limit=0.10),
        albumentations.Normalize(),
        ToTensorV2()
        ])

    dataset = dataset.Dataset(root='./semantic_drone_dataset', train=True, small=True, transforms=data_transforms)
    numtrain = int(len(dataset) * 0.7)
    numval = len(dataset) - numtrain
    trainset, valset = torch.utils.data.random_split(dataset, [numtrain, numval])
    batch_size = 8
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True )
    val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True )

    dataloaders = {"train": train_loader, "val": val_loader}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #device = torch.device("cpu")

    model = smp.Unet('resnet101', classes=20, activation=None) #, activation='softmax2d') # in pytorch, nn.CrossEntropy computes softmax
    def _enablegrad(n, b):
        for param in n.parameters():
            param.require_grad = b
    def freeze(n):
        _enablegrad(n, True)
    def unfreeze(n):
        _enablegrad(n, False)

    freeze(model.encoder)
    freeze(model.decoder)

    criterion = torch.nn.CrossEntropyLoss()

    params = [p for p in model.parameters() if p.requires_grad]
    #optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    optimizer = torch.optim.Adam(params)


In [10]:
#model(torch.randn([8,3,512,512]))

tensor([[[[-3.1658e-02,  7.4745e-02,  3.1966e-01,  ..., -1.6859e-01,
           -5.5685e-02,  3.5468e-01],
          [-2.5238e-01,  2.2679e-01,  6.0879e-02,  ...,  3.4725e-01,
            1.2959e-01,  5.6732e-01],
          [-3.2851e-01,  3.4283e-01,  1.6635e-02,  ..., -7.4110e-02,
           -3.0870e-01,  1.7543e-01],
          ...,
          [-3.2395e-01, -3.5386e-01, -2.6951e-01,  ...,  2.1741e-01,
            4.0827e-01,  3.2186e-02],
          [-5.4186e-01, -3.3302e-01, -3.8724e-01,  ...,  1.5919e-01,
            3.0003e-01,  1.1438e-01],
          [-1.5236e-01, -2.8470e-01, -4.7411e-01,  ..., -1.2138e-01,
           -1.2397e-01, -6.4874e-02]],

         [[ 6.5229e-01,  9.6220e-01,  6.0251e-01,  ...,  5.4876e-01,
            5.7102e-01,  2.3715e-01],
          [ 9.7019e-01,  9.4622e-01,  7.0076e-01,  ...,  1.3145e+00,
            9.2659e-01,  7.7206e-02],
          [ 8.1824e-01,  6.5050e-01,  9.0165e-01,  ..., -3.5225e-01,
           -6.1145e-01, -6.4392e-01],
          ...,
     